In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import time


class DetectorAPI:
    def __init__(self, path_to_ckpt):
        self.path_to_ckpt = path_to_ckpt

        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.GraphDef()
            with tf.gfile.GFile(self.path_to_ckpt, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')

        self.default_graph = self.detection_graph.as_default()
        self.sess = tf.Session(graph=self.detection_graph)

        # Definite input and output Tensors for detection_graph
        self.image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        self.detection_boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        self.detection_scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
        self.detection_classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
        self.num_detections = self.detection_graph.get_tensor_by_name('num_detections:0')

    def processFrame(self, image):
        # Expand dimensions since the trained_model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image, axis=0)
        # Actual detection.
        start_time = time.time()
        (boxes, scores, classes, num) = self.sess.run(
            [self.detection_boxes, self.detection_scores, self.detection_classes, self.num_detections],
            feed_dict={self.image_tensor: image_np_expanded})
        end_time = time.time()

        print("Elapsed Time:", end_time-start_time)

        im_height, im_width,_ = image.shape
        boxes_list = [None for i in range(boxes.shape[1])]
        for i in range(boxes.shape[1]):
            boxes_list[i] = (int(boxes[0,i,0] * im_height),
                        int(boxes[0,i,1]*im_width),
                        int(boxes[0,i,2] * im_height),
                        int(boxes[0,i,3]*im_width))
        final_score = np.squeeze(scores)
        count = 0
        for i in range(100):
            if scores is None or final_score[i]>0.5:
                count=count+1
        print(count)

        return boxes_list, scores[0].tolist(), [int(x) for x in classes[0].tolist()], int(num[0])

    def close(self):
        self.sess.close()
        self.default_graph.close()

if __name__ == "__main__":
    model_path = 'ssd_mobilenet_v1_coco_11_06_2017/frozen_inference_graph.pb'
    odapi = DetectorAPI(path_to_ckpt=model_path)
    threshold = 0.7
    cap = cv2.VideoCapture(0)

    while True:
        r, img = cap.read()
        img = cv2.resize(img, (1280, 960))

        boxes, scores, classes, num = odapi.processFrame(img)

        # Visualization of the results of a detection.

        for i in range(len(boxes)):
            # Class 1 represents human
            if classes[i] == 1 and scores[i] > threshold:
                box = boxes[i]
                cv2.rectangle(img,(box[1],box[0]),(box[3],box[2]),(255,0,0),2)

        
        cv2.imshow("preview", img)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            cap.release()
            break

Elapsed Time: 4.042235612869263
0
Elapsed Time: 0.10172176361083984
0
Elapsed Time: 0.10471725463867188
0
Elapsed Time: 0.09973287582397461
0
Elapsed Time: 0.09275197982788086
0
Elapsed Time: 0.08676648139953613
0
Elapsed Time: 0.09075784683227539
0
Elapsed Time: 0.08977246284484863
0
Elapsed Time: 0.09075784683227539
0
Elapsed Time: 0.08876299858093262
0
Elapsed Time: 0.0907588005065918
0
Elapsed Time: 0.09794735908508301
0
Elapsed Time: 0.09476447105407715
0
Elapsed Time: 0.08878016471862793
0
Elapsed Time: 0.08876323699951172
0
Elapsed Time: 0.08776545524597168
0
Elapsed Time: 0.08975982666015625
0
Elapsed Time: 0.08876276016235352
0
Elapsed Time: 0.0907585620880127
0
Elapsed Time: 0.08876276016235352
0
Elapsed Time: 0.09275221824645996
1
Elapsed Time: 0.08776617050170898
1
Elapsed Time: 0.09275436401367188
1
Elapsed Time: 0.08776473999023438
1
Elapsed Time: 0.08976054191589355
1
Elapsed Time: 0.08776521682739258
1
Elapsed Time: 0.08976054191589355
1
Elapsed Time: 0.0877654552459716